In [73]:
import pyreadstat
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [74]:
#load all files

don=pd.read_csv("../1_data/private/donation_donor2.csv")
defr=pd.read_csv("../1_data/private/deferral.csv")
collect=pd.read_csv("../1_data/private/collec_sites.csv")
med_quest=pd.read_csv("../1_data/private/medical_quest.csv")
drive=pd.read_csv("../1_data/private/drive.csv")

#dimensions
print(don.shape)
print(defr.shape)

(5715779, 35)
(1084380, 17)


In [75]:
don_hist=pd.read_csv("../1_data/private/don_hist2_V.csv")

In [76]:
# merge don with collect and drives

collect=pd.read_csv("../1_data/private/collec_sites.csv", error_bad_lines=False) 
don= pd.merge(don, collect, how='left', on=["COLLECTION_SITE_KEY"])
drive=pd.read_csv("../1_data/private/drive.csv", error_bad_lines=False)
don=pd.merge(don, drive, how='left', on=["DRIVE_ID"])

In [77]:
#merge donor history
#merge donor_history
don['DonorID']=don['DONOR_KEY']
don["Visit_Date"] = pd.to_datetime(don['DON_DATE_KEY'], format='%Y%m%d') #convert to datetime object
don['Visit_Date']=pd.to_datetime(don['Visit_Date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ') #covert to ISO -#2019-03-26T00:00:00Z
#don.rename(columns={'DONATION_NUMBER_x': 'DONATION_NUMBER'}, inplace=True)

don=pd.merge(don, don_hist[[ 'DonorID', 'Visit_Date','cum_lifetime_donations',
                          'rbc_loss_last_12_months', 'rbc_loss_last_24_months']], how='left', on=["DonorID", "Visit_Date"])


In [78]:
#preprocess med_quest

med_questHGB=med_quest.loc[(med_quest['QUESTION_CODE']=='5U')] #df with hgb values
print(len(med_questHGB))
med_questHGB = med_questHGB.drop_duplicates()
print(len(med_questHGB))

med_questHGB_recheck=med_quest.loc[(med_quest['QUESTION_CODE']=='5V')] #df with hgb recheck values
print(len(med_questHGB_recheck))
med_questHGB_recheck=med_questHGB_recheck.drop_duplicates()
print(len(med_questHGB_recheck))


6564933
6564877
551256
551256


In [79]:
#define hgb value-> if recheck if not na replace hgb value with recheck value
m=pd.merge(med_questHGB[["DON_DATE_KEY", "DONATION_NUMBER", 'MQ_ANSWER']], med_questHGB_recheck[[ "DON_DATE_KEY", "DONATION_NUMBER", 'MQ_ANSWER']], how='left', on=['DONATION_NUMBER']) 
print(len(m))
m=pd.merge(m, med_questHGB_recheck, how='left', on=['DONATION_NUMBER'])
m = m.drop_duplicates()
print(len(m))
hgb_dups = m[m.duplicated(subset='DONATION_NUMBER', keep=False)]
m = m[~m['DONATION_NUMBER'].isin(hgb_dups['DONATION_NUMBER'])]
print(len(m))


6564881
6564889
6564803


In [80]:
#fix errors
m.loc[m.MQ_ANSWER_y=='NOT RECHECKED', 'MQ_ANSWER_y'] = np.NaN
m.loc[m.MQ_ANSWER_x=='NO RESPONSE', 'MQ_ANSWER_x'] = np.NaN
m.loc[m.MQ_ANSWER_x=='WALKOUT', 'MQ_ANSWER_x'] = np.NaN
m.loc[m.MQ_ANSWER_x=='Not Performed', 'MQ_ANSWER_x'] = np.NaN
m.loc[m.MQ_ANSWER=='NOT RECHECKED', 'MQ_ANSWER'] = np.NaN
m['5V']=pd.to_numeric(m['MQ_ANSWER_y'])
m['5U']=pd.to_numeric(m['MQ_ANSWER_x'])


In [81]:
m['HGB_value']= m[["5U", "5V"]].max(axis=1)
#m['HGB_value']=np.where((m['5V'].notna()), m['5V'], m['5U'])#

m.drop(['MQ_ANSWER', 'MQ_ANSWER_x', 'MQ_ANSWER_y'], axis=1, inplace=True)

m['HGB_value'].replace(np.nan, 999, regex=True, inplace=True)

#add hgb values
df=pd.merge(don, m[["DONATION_NUMBER", 'HGB_value']], how='left', on=["DONATION_NUMBER"]) 


In [82]:
m.DONATION_NUMBER.nunique()

6564803

In [83]:
len(don)

5715779

In [84]:
len(df)

5715779

In [85]:
df1=pd.merge(df, defr, how='left', on=["DONOR_KEY", "DON_DATE_KEY"])
print(len(df1))

5769588


In [86]:
#dfsub.to_csv(("../1_data/private/sub_data.csv"), index=False)
df1.to_csv('../1_data/private/VitalantData.csv', index=False)